## Dolly 3B fine tuning through PEFT/LoRA - Inference

In [ ]:
#!pip install transformers
#!pip install datasets
#!pip install gradio
#!pip install py7zr
#!pip install accelerate
#!pip install bitsandbytes
#!pip install peft
#!pip install rouge_score
#!pip install evaluate

#!pip install --upgrade accelerate

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling, DataCollatorForSeq2Seq

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
import pandas as pd
import numpy as np
import time
import re, os
import gc

### Load LoRA model

In [ ]:
BASE_MODEL = 'databricks/dolly-v2-3b'
OUTPUT_DIR = 'models/dolly3b-loraft-r4'

In [ ]:
device_map = "auto"

In [ ]:
from peft import PeftModel, PeftConfig
config = PeftConfig.from_pretrained(OUTPUT_DIR)
config

In [ ]:
tokenizer_ft = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer_ft.pad_token = tokenizer_ft.eos_token
tokenizer_ft.add_special_tokens({"additional_special_tokens": [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL]})

In [ ]:
model_ft = AutoModelForCausalLM.from_pretrained(  
    config.base_model_name_or_path,  
    load_in_8bit=True,
    device_map=device_map,
    #cache_dir="/LLM_test/hf_cache"
)

In [ ]:
model_ft = PeftModel.from_pretrained(
    model_ft, 
    OUTPUT_DIR, 
    device_map=device_map,
)

In [ ]:
model_ft.resize_token_embeddings(len(tokenizer_ft))

In [ ]:
!nvidia-smi

### Inference using instruct pipeline

In [ ]:
# Create Instruct Pipeline
import logging
import re

import numpy as np
from transformers import Pipeline, PreTrainedTokenizer

logger = logging.getLogger(__name__)

# This is the prompt that is used for generating responses using an already trained model.  It ends with the response
# key, where the job of the model is to provide the completion that follows it (i.e. the response itself).
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

def get_special_token_id(tokenizer: PreTrainedTokenizer, key: str) -> int:
    """Gets the token ID for a given string that has been added to the tokenizer as a special token.
    When training, we configure the tokenizer so that the sequences like "### Instruction:" and "### End" are
    treated specially and converted to a single, new token.  This retrieves the token ID each of these keys map to.
    Args:
        tokenizer (PreTrainedTokenizer): the tokenizer
        key (str): the key to convert to a single token
    Raises:
        RuntimeError: if more than one ID was generated
    Returns:
        int: the token ID for the given key
    """
    token_ids = tokenizer.encode(key)
    if len(token_ids) > 1:
        raise ValueError(f"Expected only a single token for '{key}' but found {token_ids}")
    return token_ids[0]


class InstructionTextGenerationPipeline(Pipeline):
    def __init__(
        self, *args, do_sample: bool = True, max_new_tokens: int = 256, top_p: float = 0.92, top_k: int = 0, **kwargs
    ):
        super().__init__(*args, do_sample=do_sample, max_new_tokens=max_new_tokens, top_p=top_p, top_k=top_k, **kwargs)

    def _sanitize_parameters(self, return_instruction_text=False, **generate_kwargs):
        preprocess_params = {}

        # newer versions of the tokenizer configure the response key as a special token.  newer versions still may
        # append a newline to yield a single token.  find whatever token is configured for the response key.
        tokenizer_response_key = next(
            (token for token in self.tokenizer.additional_special_tokens if token.startswith(RESPONSE_KEY)), None
        )

        response_key_token_id = None
        end_key_token_id = None
        if tokenizer_response_key:
            try:
                response_key_token_id = get_special_token_id(self.tokenizer, tokenizer_response_key)
                end_key_token_id = get_special_token_id(self.tokenizer, END_KEY)

                # Ensure generation stops once it generates "### End"
                generate_kwargs["eos_token_id"] = end_key_token_id
            except ValueError:
                pass

        forward_params = generate_kwargs
        postprocess_params = {
            "response_key_token_id": response_key_token_id,
            "end_key_token_id": end_key_token_id,
            "return_instruction_text": return_instruction_text,
        }

        return preprocess_params, forward_params, postprocess_params

    def preprocess(self, instruction_text, **generate_kwargs):
        prompt_text = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction_text)
        inputs = self.tokenizer(
            prompt_text,
            return_tensors="pt",
        )
        inputs["prompt_text"] = prompt_text
        inputs["instruction_text"] = instruction_text
        return inputs

    def _forward(self, model_inputs, **generate_kwargs):
        input_ids = model_inputs["input_ids"]
        attention_mask = model_inputs.get("attention_mask", None)
        generated_sequence = self.model.generate(
            input_ids=input_ids.to(self.model.device),
            attention_mask=attention_mask,
            pad_token_id=self.tokenizer.pad_token_id,
            **generate_kwargs,
        )[0].cpu()
        instruction_text = model_inputs.pop("instruction_text")
        return {"generated_sequence": generated_sequence, "input_ids": input_ids, "instruction_text": instruction_text}

    def postprocess(self, model_outputs, response_key_token_id, end_key_token_id, return_instruction_text):
        sequence = model_outputs["generated_sequence"]
        instruction_text = model_outputs["instruction_text"]

        # The response will be set to this variable if we can identify it.
        decoded = None

        # If we have token IDs for the response and end, then we can find the tokens and only decode between them.
        if response_key_token_id and end_key_token_id:
            # Find where "### Response:" is first found in the generated tokens.  Considering this is part of the
            # prompt, we should definitely find it.  We will return the tokens found after this token.
            response_pos = None
            response_positions = np.where(sequence == response_key_token_id)[0]
            if len(response_positions) == 0:
                logger.warn(f"Could not find response key {response_key_token_id} in: {sequence}")
            else:
                response_pos = response_positions[0]

            if response_pos:
                # Next find where "### End" is located.  The model has been trained to end its responses with this
                # sequence (or actually, the token ID it maps to, since it is a special token).  We may not find
                # this token, as the response could be truncated.  If we don't find it then just return everything
                # to the end.  Note that even though we set eos_token_id, we still see the this token at the end.
                end_pos = None
                end_positions = np.where(sequence == end_key_token_id)[0]
                if len(end_positions) > 0:
                    end_pos = end_positions[0]

                decoded = self.tokenizer.decode(sequence[response_pos + 1 : end_pos]).strip()
        else:
            # Otherwise we'll decode everything and use a regex to find the response and end.

            fully_decoded = self.tokenizer.decode(sequence)

            # The response appears after "### Response:".  The model has been trained to append "### End" at the
            # end.
            m = re.search(r"#+\s*Response:\s*(.+?)#+\s*End", fully_decoded, flags=re.DOTALL)

            if m:
                decoded = m.group(1).strip()
            else:
                # The model might not generate the "### End" sequence before reaching the max tokens.  In this case,
                # return everything after "### Response:".
                m = re.search(r"#+\s*Response:\s*(.+)", fully_decoded, flags=re.DOTALL)
                if m:
                    decoded = m.group(1).strip()
                else:
                    logger.warn(f"Failed to find response in:\n{fully_decoded}")

        if return_instruction_text:
            return {"instruction_text": instruction_text, "generated_text": decoded}

        return decoded
     

In [ ]:
dolly_gen = InstructionTextGenerationPipeline(model=model_ft, tokenizer=tokenizer_ft)


Single query test

In [ ]:
instruction = "Do I need to look out for an email with instructions?"

response = dolly_gen(instruction)
response

### Batch inference test and evaluation

In [ ]:
import pandas as pd
import time

In [ ]:
INPUT_FILE = 'mlu_ops_concat_all_df_list_instruction_output_1441.json'
OUTPUT_FILE = 'responses_dolly3bloraft_r4inf.csv'

In [ ]:
import pandas as pd
df_results = pd.read_json(INPUT_FILE)
df_results = df_results.sample(n=200, random_state=123)
prompt_list = df_results.instruction.values.tolist()


In [ ]:
response_list = []

newline, bold, unbold = "\n", "\033[1m", "\033[0m"

st = time.time()

i=0
for text in prompt_list:
    print(i," ",end = '')
    response = dolly_gen(text)
    response_list.append(response)
    i = i+1

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
df_response = pd.DataFrame()  
df_response["prompt"] = prompt_list
df_response["dolly3bloraft-r40"] = response_list
df_response.to_csv(OUTPUT_FILE, index=False)

Performance evaluation via ROUGE SCORE

In [ ]:
from rouge_score import rouge_scorer
import evaluate
rouge = evaluate.load('rouge')
import numpy as np

In [ ]:
def calculate_rouge(pred_list,ref_list):
    
    rouge_results = rouge.compute(predictions=pred_list,
                         references=ref_list,
                         use_aggregator=True)
    avg_rougeLsum = np.mean(rouge_results["rougeLsum"])
    avg_rougeL = np.mean(rouge_results["rougeL"])
    avg_rouge2 = np.mean(rouge_results["rouge2"])
    avg_rouge1 = np.mean(rouge_results["rouge1"])
    
    print("Average rouge scores: ", avg_rougeLsum, avg_rougeL, avg_rouge2, avg_rouge1)
    
    return 

In [ ]:
ref_list = list(df_results["output"])
dolly3b_list = list(df_response["dolly3bloraft-r10"])

In [ ]:
calculate_rouge(dolly3b_list, ref_list)            